In [ ]:
import sys
import os
import argparse
import logging
import shutil
import re
import random
import pickle
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
from torchvision.datasets import VisionDataset
from torchvision import transforms
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, auc, roc_curve
from collections import Counter
from utils import *
from auto_tqdm import tqdm
import augmentations
from sklearn.model_selection import train_test_split
from utils import balanced_dataset
from PIL import ImageColor
from matplotlib.pyplot import figure

import sys
sys.path.append("../clinical_data_classifier")
from rtog_helper import rtog_from_study_number
from histopathology_image_helper import CaseManager
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.manifold import TSNE
def str2bool(v):
    if isinstance(v, bool):
       return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


In [ ]:
cm = CaseManager()


In [ ]:
slideInfo_9408 = [
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/11.10.2020/9408_11.10.2020 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/11.23.2020/9408_11.23.2020 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/12.8.2020/9408_12.8.2020 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/12.22.2020/9408_12.22.2020 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/1.12.2021/9408_1.12.2021 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/1.26.2021/9408_1.26.2021 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/2.16.2021/9408_2.16.2021 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/3.2.2021/9408_3.2.2021 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/3.16.2021/9408_3.16.2021 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/3.30.2021/9408_3.30.2021 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/4.13.2021/9408_4.13.2021 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/4.27.2021/9408_4.27.2021 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/5.11.2021/9408_5.11.2021 DeIDed Slide Information.xlsx',
    '/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/5.13.2021 all rescans/9408_5.13.2021 DeIDed Slide Information.xlsx',
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/6.9.2021/9408_6.9.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/7.28.2021/9408_7.28.2021 DE-ID'ed Export.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9408/8.3.2021 Histo/9408_8.3.2021-Deided- Histo Export.xlsx"
]

slideInfo_9202 = [
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/1.12.2021/9202_1.12.2021 DeIDed Slide Information updated 2.4.2021.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/1.26.2021/9202_1.26.2021 DeIDed Slide Information updated 2.4.2021.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/10.21.2020/9202_10.21.2020 DeID Slide Information.xlsx" ,
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/10.5.2020/9202_10.5.2020 DeIDed Slide Information Updated 10.21.20.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/11.24.2020/9202_11.24.2020 DeIDed Slide Information updated 12.9.2020.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/11.9.2020/9202_11.9.2020 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/12.22.2020/9202_12.22.2020 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/12.8.2020/9202_12.8.2020 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/2.16.2021/9202_2.16.2020 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/3.16.2021/9202_3.16.2020 DeIDed Slide Information updated 4.5.2021.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/4.27.2021/9202_4.27.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/5.11.2021 all rescans/9202_5.11.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/5.25.2021/9202_5.25.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/9.14.2020/9202_9.14.2020 DeIDed Slide Information Updated 10.21.20.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/8.3.2021/9202_Upload_8.3.2021 De-ided.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9202/8.3.2021 Histo/9202 Histoslides_Deided-8.3.2021.xlsx"
]

slideInfo_9413 = [
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9413/10.22.2020/9413_10.22.2020 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9413/10.5.2020/9413_10.5.2020 DeIDed Slide Information Updated 10.21.20.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9413/11.24.2020/9413_11.24.2020 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9413/11.9.2020 RC slides/9413_11.9.2020 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9413/8.25.2020/8.25.2020 DeIDed Slide Information Updated 10.21.2020.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9413/5.11.2021 3 rescans/9413_5.11.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9413/6.9.2021/9413_6.9.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9413/7.28.2021 Histo/De-ided-9413_7.28.2021 Histology Export.xlsx" 
]

slideInfo_9910 = [
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9910/6.9.2021/9910_6.9.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9910/6.21.2021/9910_6.21.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9910/7.21.2021 Blurry/9910_7.21.2021_Blurry DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9910/7.28.2021/Deided-9910_7.28.2021 Export.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-9910/8.10.2021/9910-8.11.2021_deid-upload.xlsx"
]


slideInfo_0126 = [
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-0126/6.30.2021/0126_6.30.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-0126/7.21.2021/0126_7.21.2021 DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-0126/7.21.2021 Blurry/0126_7.21.2021 Blurry DeIDed Slide Information.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-0126/7.28.2021/0126_7.28.2021 DE-ID'ed Export.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-0126/8.3.2021/0126_8.3.21 De-id-Export.xlsx",
    "/export/medical_ai/ucsf/box_data/Aperio Images of NRG GU H&E Slides/RTOG-0126/8.10.2021/0126-8.10.2021-Deided-upload.xlsx"
]

def get_df_list(slideInfoList):
    df_list = []
    for name in slideInfoList:
        df = pd.read_excel(name, engine='openpyxl')
        
        if "Unnamed: 2" in df.columns:
            df.rename(columns=df.iloc[0], inplace = True)
            df.drop([0], inplace = True)
        if "CN_deidentified" in df.columns:
            df = df.rename(columns={"CN_deidentified":"cn_deidentified"})
        if "CN_DeIdentified" in df.columns:
            df = df.rename(columns={"CN_DeIdentified":"cn_deidentified"})
        if "CN_Deidentified" in df.columns:
            df = df.rename(columns={"CN_Deidentified":"cn_deidentified"})
        if "CN Deid" in df.columns:
            df = df.rename(columns={"CN Deid":"cn_deidentified"}) 
        if "CN-DeID'ed" in df.columns:
            df = df.rename(columns={"CN-DeID'ed":"cn_deidentified"}) 
        if "CN DE-ID" in df.columns:
            df = df.rename(columns={"CN DE-ID":"cn_deidentified"}) 
        if "CN deidentified" in df.columns:
            df = df.rename(columns={"CN deidentified":"cn_deidentified"})
        if "CN De-Identified" in df.columns:
            df = df.rename(columns={"CN De-Identified":"cn_deidentified"})  
        if "CN De-ID" in df.columns:
            df = df.rename(columns={"CN De-ID":"cn_deidentified"})           
        
        # Two excel files have the Treatment column named differently
        if "Timepoint" in df.columns:
            df = df.rename(columns={"Timepoint":"Treatment"})
        if "Tx" in df.columns:
            df = df.rename(columns={"Tx":"Treatment"})
            
        df = df.rename(columns={"Image ID": "image_id"})
        df = df.dropna(subset=['cn_deidentified'])

        df["image_id"] = df["image_id"].apply(lambda x: str(int(x)) + ".pkl")
        df["cn_deidentified"] = df["cn_deidentified"].apply(lambda x: int(x))
        
        treatment_list = ['Pre-2', 'Pre-3', 'pre', 'pre-1', 'Pre', 'Pre-1']
        df = df[df['Treatment'].isin(treatment_list)]
        df_list += [df]
    return df_list

slide_info_df_9408 = get_df_list(slideInfo_9408)
slide_info_df_9413 = get_df_list(slideInfo_9413)
slide_info_df_9202 = get_df_list(slideInfo_9202)
slide_info_df_9910 = get_df_list(slideInfo_9910)
slide_info_df_0126 = get_df_list(slideInfo_0126)

In [ ]:
import random

def get_study(feature_path, study, slide_info_dfs, case_df):
    tissue_path = "/export/medical_ai/ucsf/tissue_vs_non_pkl/v2/RTOG-{}/".format(study)
    unique_cn = []
    for df in slide_info_dfs:            
        unique_cn += list(df.cn_deidentified.unique())
    unique_cn = list(set(unique_cn))
    features = []
    features_full = []
    paths_full = []
    for cn in tqdm(unique_cn[::-1]):
        if not cn in case_df.cn_deidentified.unique():
            continue
        slides = []
        for df in slide_info_dfs:
            slides += list(df.loc[df.cn_deidentified == cn].image_id)

    #     print(slides, cn)
        for slide in slides:
            if not os.path.exists(feature_path + slide):
                continue
            data = pickle.load(open(feature_path + slide, "rb"))
            random.shuffle(data)
            features += [data[:int(len(data)/4)]]
            features_full += [data]
            df = pd.read_pickle(tissue_path + slide)
            df = df.loc[df.tissue_vs_non]
            paths_full += [list(df.path)]
    features = np.concatenate(features)
    features_full = np.concatenate(features_full)
    paths_full = np.concatenate(paths_full)
    return features, features_full, paths_full

# feature_path = "/export/medical_ai/ucsf/ssl_rtog/moco/model_R50_b=256_lr=0.03_pg4plus_imagenet_pretrained_nucleic/features/RTOG-9202_features/"
# features, features_full, paths_full = get_study(feature_path, "9202", slide_info_dfs_9202, case_df_9202)
# feature_path = "/export/medical_ai/ucsf/ssl_rtog/moco/model_R50_b=256_lr=0.03_pg4plus_imagenet_pretrained_nucleic/features/RTOG-9413_features/"
# features, features_full, paths_full = get_study(feature_path, "9413", slide_info_dfs_9413, case_df_9413)
feature_path = "/export/medical_ai/ucsf/ssl_rtog/moco/model_R50_b=256_lr=0.03_pg4plus_imagenet_pretrained_nucleic/features/RTOG-9408_features/"
features, features_full, paths_full = get_study(feature_path, "9408", slide_info_dfs_9408, case_df_9408)


In [ ]:
# get all data
feature_path = "/export/medical_ai/ucsf/ssl_rtog/moco/model_R50_b=256_lr=0.03_pg4plus_imagenet_pretrained_nucleic/features/RTOG-9202_features/"
features_9202, features_full_9202, paths_full_9202 = get_study(feature_path, "9202", slide_info_dfs_9202, case_df_9202)
feature_path = "/export/medical_ai/ucsf/ssl_rtog/moco/model_R50_b=256_lr=0.03_pg4plus_imagenet_pretrained_nucleic/features/RTOG-9413_features/"
features_9413, features_full_9413, paths_full_9413 = get_study(feature_path, "9413", slide_info_dfs_9413, case_df_9413)
feature_path = "/export/medical_ai/ucsf/ssl_rtog/moco/model_R50_b=256_lr=0.03_pg4plus_imagenet_pretrained_nucleic/features/RTOG-9408_features/"
features_9408, features_full_9408, paths_full_9408 = get_study(feature_path, "9408", slide_info_dfs_9408, case_df_9408)
features = np.concatenate([features_9202, features_9413, features_9408])
features_full = np.concatenate([features_full_9202, features_full_9413, features_full_9408])
paths_full = np.concatenate([paths_full_9202, paths_full_9413, paths_full_9408])

In [ ]:
clf = KMeans(n_clusters=25)
clf.fit(features)
cluster_labels = clf.predict(features_full)

In [ ]:
from sklearn.metrics import pairwise_distances
distances = pairwise_distances(clf.cluster_centers_, features_full)
argsorted_distances = np.argsort(distances)

In [ ]:
plot_paths = []
for cluster in argsorted_distances:
    cluster_paths = []
    for idx in cluster[:25]:
        cluster_paths += [paths_full[idx]]
    plot_paths += [cluster_paths]
    
    
empty_img = np.zeros((256*25 + 25*10, 256*25 + 25*10, 3)).astype(np.uint8)
for i, cluster in enumerate(plot_paths):
    for j, img in enumerate(cluster):
        x = i * 256 + i * 10
        y = j * 256 + j * 10
        img = Image.open(img)
        empty_img[x:x+256, y:y+256] = np.array(img).astype(np.uint8)
plt.imshow(empty_img)
save_path = "./cluster_viz/umap_nucleic/"
plt.imsave(save_path + "all_studies_kmeans_25_clust.png", empty_img)


In [ ]:
# plot extended data figure with numbers in the sidebar. 
from PIL import ImageDraw, Image

plot_paths = []
for cluster in argsorted_distances:
    cluster_paths = []
    for idx in cluster[:25]:
        cluster_paths += [paths_full[idx]]
    plot_paths += [cluster_paths]
    
    
empty_img = np.zeros((256*25 + 26*10,256 + 256*25 + 25*10, 3)).astype(np.uint8)
for i, cluster in enumerate(plot_paths):
    for j, img in enumerate(cluster):
        x = i * 256 + i * 10 + 10
        y = 256 + j * 256 + j * 10
        img = Image.open(img)
        empty_img[x:x+256, y:y+256] = np.array(img).astype(np.uint8)
    if i == 5:
        break


img = Image.fromarray(empty_img)
draw = ImageDraw.Draw(img)
font = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", 200)
font_small = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", 12)
for i, _ in enumerate(plot_paths):
    for j, _ in enumerate(cluster):
        x = i * 256 + i * 10
        y = 256 + j * 256 + j * 10
        draw.text((y - 10,x-3), str(j), (255,255,255),font=font_small)


for i in range(len(plot_paths)):
    draw.text((0,i*256 + i*10), str(i), (255,255,255),font=font)

plt.imshow(np.array(img))
# plt.imshow(empty_img)
save_path = "./cluster_viz/umap_nucleic/"
plt.imsave(save_path + "all_studies_kmeans_25_clust_num_test.png", np.array(img))


In [ ]:
colors = ["#696969", "#FF5733", "#8b4513","#483d8b","#008000","#000080","#9acd32","#daa520","#8b008b","#00ced1","#ffff00","#7cfc00","#00fa9a","#8a2be2","#dc143c","#00bfff","#0000ff","#d8bfd8","#ff00ff","#1e90ff","#db7093","#ff1493","#ffa07a","#ee82ee","#ffffe0"]
colors = [ImageColor.getcolor(x, "RGB") for x in colors]

In [ ]:
import umap.umap_
indices = np.random.choice(len(features), 60000)

projection_vectors = umap.umap_.UMAP(n_neighbors=15, min_dist=0.3).fit_transform(features_full[indices])
plot_path = "cluster_viz/umap_nucleic/all_studies_scatter.png"

plot_scatter(plot_path, projection_vectors, cluster_labels[indices], colors, "all_studies umap 25 clusters no SVD", save_fig=True)

In [ ]:
reconstruction = {"projection_vectors": projection_vectors,
                    "cluster_labels": cluster_labels[indices],
                    "data_vectors": features[indices],
                    "image_list":  paths_full[indices],
                  "cluster_centers": clf.cluster_centers_,
                 }
import pickle
# with open("/export/home/prostate/precision_main3/SSL/simclr/cluster_viz/umap_nucleic/all_studies.pkl", "wb") as f:
#     pickle.dump(reconstruction, f)

In [ ]:
# download e2i package here: https://github.com/YontiLevin/Embeddings2Image
# replace modules.py with this file: https://github.com/MetaMind/douwe-scratchpad/tree/master/precision_oncology/embedding2img
# do setup.py install
from e2i import EmbeddingsProjector

In [ ]:
# cluster_indices = np.random.choice(len(features), 150000)

# clf = KMeans(n_clusters=25)#[KMeans(n_clusters=i) for i in Ks]
# clf.fit(features[cluster_indices])

def plot_visual_tsne(plot_path, features, paths, clf, colors):
    indices = np.random.choice(len(features), 50000)
    cluster_labels = clf.predict(features[indices])

    image = EmbeddingsProjector()
    image.output_img_size =  15000
    image.each_img_size = 256
    image.colors = colors
    image.cluster_labels = cluster_labels
    image.data_vectors = features[indices]
    image.image_list = paths[indices]
#     image._svd=False
    image._crop()
    
    print("starting calculation")
    image.calculate_projection()
    print("creating image")
#     image.create_image(plot_path)
    return image
plot_path = "./test_no_svd.jpg"
image = plot_visual_tsne(plot_path, features, paths_full, clf, colors)

# plot embeddings

In [ ]:
import umap.umap_
indices = np.random.choice(len(features), 50000)

projection_vectors = umap.umap_.UMAP(n_neighbors=15, min_dist=0.3).fit_transform(features_full[indices])


In [ ]:
plot_path = "./trash"
# 5 neighbours
plot_scatter(plot_path, projection_vectors, cluster_labels[indices], colors, "9202 umap 25 clusters no SVD")

In [ ]:
plot_path = "./trash"
#10 neighbours
plot_scatter(plot_path, projection_vectors, cluster_labels[indices], colors, "9202 umap 25 clusters no SVD")

In [ ]:
plot_path = "./trash"
#15 neighbours
plot_scatter(plot_path, projection_vectors, cluster_labels[indices], colors, "9202 umap 25 clusters no SVD")

In [ ]:
plot_path = "./trash"
#20 neighbours
plot_scatter(plot_path, projection_vectors, cluster_labels[indices], colors, "9202 umap 25 clusters no SVD")

In [ ]:

def plot_scatter(plot_path, projection_vectors, cluster_labels, colors, title="", save_fig=False):
    figure(figsize=(16, 9), dpi=1000)
    for class_id in list(set(cluster_labels)):
        bool_arr = cluster_labels == class_id
        to_plot = projection_vectors[bool_arr]
        plt.scatter(to_plot[:,0][:200], to_plot[:,1][:200], c=np.array([colors[class_id]])/255)
    if save_fig:
        plt.savefig(plot_path)
    plt.title(title)
    plt.show()
plot_path = "./test_scatter_no_svd.jpg"
plot_scatter(plot_path, image.projection_vectors, image.cluster_labels, colors, "with svd 9202")

# save embeddings for later

In [ ]:
reconstruction = {"projection_vectors": image.projection_vectors,
                    "cluster_labels": cluster_labels,
                    "data_vectors": features[indices],
                    "image_list":  paths[indices],
                  "cluster_centers": clf.cluster_centers_,
                 }
import pickle
# with open("/export/home/prostate/precision_main3/SSL/simclr/cluster_viz/nucleic_with_saves/all_studies_data.pkl", "wb") as f:
#     pickle.dump(reconstruction, f)

In [ ]:
with open("/export/home/prostate/precision_main3/SSL/simclr/cluster_viz/nucleic_with_saves/9202_data.pkl", "rb") as f:
    recon_9202 = pickle.load(f)

In [ ]:
plot_path = "./test_scatter.jpg"
colors_find_cluster = [(0,0,0)] * 25
colors_find_cluster[13]=(255,0,0)
plot_scatter(plot_path, recon_9202["projection_vectors"], recon_9202["cluster_labels"], colors_find_cluster)

In [ ]:
for i in range(25):
    plot_path = "./test_scatter.jpg"
    colors_find_cluster = [(0,0,0)] * 25
    colors_find_cluster[i]=(255,0,0)
    print("plotting: ", i)
    plot_scatter(plot_path, recon_9202["projection_vectors"], recon_9202["cluster_labels"], colors_find_cluster)

# extended data figures plots

In [ ]:
plot_path = "./test_scatter.jpg"
colors_find_cluster = [(128,128,128)] * 25
colors_find_cluster[4]=(255,0,0)
colors_find_cluster[5]=(0,255,0)
colors_find_cluster[10]=(0,0,255)
colors_find_cluster[12]=(255,255,0)
colors_find_cluster[16]=(255,0,255)
colors_find_cluster[21]=(0,255,255)

plot_scatter(plot_path, recon_9202["projection_vectors"], recon_9202["cluster_labels"], colors_find_cluster)

In [ ]:
plot_path = "./test_scatter.jpg"
colors_find_cluster = [(128,128,128)] * 25
colors_find_cluster[8]=(255,0,0)
colors_find_cluster[11]=(0,255,0)
colors_find_cluster[13]=(0,0,255)
colors_find_cluster[17]=(255,255,0)
colors_find_cluster[16]=(255,0,255)
colors_find_cluster[23]=(0,255,255)

plot_scatter(plot_path, projection_vectors, cluster_labels[indices], colors_find_cluster)

In [ ]:
# find which cluster is located where. 
for i in range(25):
    plot_path = "./test_scatter.jpg"
    colors_find_cluster = [(0,0,0)] * 25
    colors_find_cluster[i]=(255,0,0)
    print("plotting: ", i)
    plot_scatter(plot_path, projection_vectors, cluster_labels[indices], colors_find_cluster)